<a href="https://colab.research.google.com/github/Annamjohn/Twitter-Sentiment-Analysis/blob/main/Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install kaggle library
! pip install kaggle

Upload your kaggle.json file

In [5]:
# configure the path of kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [85]:
# fetch and import Twitter Sentiment dataset using API
!kaggle datasets download -d bittlingmayer/amazonreviews

Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
 99% 490M/493M [00:09<00:00, 85.2MB/s]
100% 493M/493M [00:09<00:00, 57.2MB/s]


In [86]:
#Extract the compressed dataset

from zipfile import ZipFile
dataset= '/content/amazonreviews.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("The dataset has been extracted")

The dataset has been extracted


In [89]:
#import the dependencies
import numpy as np
import pandas as pd
import re #pattern matching, search through data
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [90]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data Processing

In [93]:
# # loading data from csv file to pd DF

twitter_data=pd.read_csv('/content/Airline-Sentiment.csv', encoding= 'ISO-8859-1')

In [94]:
#no. of rows and columns
twitter_data.shape

(14640, 20)

In [95]:
twitter_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)


In [96]:
twitter_data.isnull().sum() # sum of no. of null vals in each column



_unit_id                            0
_golden                             0
_unit_state                         0
_trusted_judgments                  0
_last_judgment_at                  56
airline_sentiment                   0
airline_sentiment:confidence        0
negativereason                   5462
negativereason:confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_id                            0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [97]:
# twitter_data['content'] = twitter_data['content'].fillna('')
#twitter_data.isnull().sum()

In [99]:
twitter_data['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

**Stemming**

In [100]:
port_stem=PorterStemmer()

In [101]:
def stemming(content):

    # Remove unwanted patterns
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # ^ =>not i.e. replace all non-alphabetic characters in content with a space
  stemmed_content= stemmed_content.lower()
  stemmed_content= stemmed_content.split() # Split the string into a list of words based on whitespace
  stemmed_content= [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content= ' '.join(stemmed_content) # Joins the list of stemmed words back into a single string for each tweet, with words separated by spaces.

  return stemmed_content

In [102]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [109]:
twitter_data= twitter_data[['stemmed_content','airline_sentiment']]
twitter_data.head()

,stemmed_content,airline_sentiment
0,virginamerica dhepburn said,neutral
1,virginamerica plu ad commerci experi tacki,positive
2,virginamerica today must mean need take anoth ...,neutral
3,virginamerica realli aggress blast obnoxi ente...,negative
4,virginamerica realli big bad thing,negative


In [110]:
print(twitter_data['stemmed_content'])

0                              virginamerica dhepburn said
1               virginamerica plu ad commerci experi tacki
2        virginamerica today must mean need take anoth ...
3        virginamerica realli aggress blast obnoxi ente...
4                       virginamerica realli big bad thing
                               ...                        
14635          americanair thank got differ flight chicago
14636    americanair leav minut late flight warn commun...
14637    americanair pleas bring american airlin blackb...
14638    americanair money chang flight answer phone su...
14639    americanair ppl need know mani seat next fligh...
Name: stemmed_content, Length: 14640, dtype: object


In [111]:
print(twitter_data['airline_sentiment'])

0         neutral
1        positive
2         neutral
3        negative
4        negative
           ...   
14635    positive
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14640, dtype: object


In [112]:
# seperate data and label

X= twitter_data['stemmed_content'].values
Y=twitter_data['airline_sentiment'].values

In [113]:
print(X) # 1st 3 and last 3

['virginamerica dhepburn said'
 'virginamerica plu ad commerci experi tacki'
 'virginamerica today must mean need take anoth trip' ...
 'americanair pleas bring american airlin blackberri'
 'americanair money chang flight answer phone suggest make commit'
 'americanair ppl need know mani seat next flight plz put us standbi peopl next flight']


In [114]:
print(Y)

['neutral' 'positive' 'neutral' ... 'neutral' 'negative' 'neutral']


In [115]:
#Split dataset into train and test set
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.05, stratify= Y, random_state=42)

In [116]:
print(X.shape, X_train.shape, X_test.shape) # 80-20

(14640,) (13908,) (732,)


In [117]:
print(X_train)

['unit min ewr baggag claim stillnobag ridicul'
 'jetblu updat pleas flight continu delay' 'usairway done' ...
 'jetblu enough stop fli jetblu' 'jetblu thank much relax flight'
 'unit learn book flight http co mhh ruplv']


In [118]:
print(X_test)

['usairway mani agent work handl thousand call'
 'southwestair u get flt instead rt' 'jetblu right'
 'usairway tell reserv rebook'
 'americanair realli appreci great custom servic one servic agent hung ask legitim question'
 'unit funni flight yesterday delay mechan issu rotten luck get plane row issu'
 'usairway dude name shaquil desk charlott incred profession help vs crazi angri peopl tonight'
 'americanair third time call hung anyon speak'
 'americanair paid seat upgrad b sever weather got book anoth flight get upgrad call inquir help'
 'virginamerica ladygaga carrieunderwood juli andrew first ladi gaga wow last night carri meh'
 'usairway umm defin extra time' 'unit thank'
 'unit cat flight delay hour arriv hawaii amp abl pick tomorrow'
 'usairway pet polici' 'unit thank'
 'southwestair list prefer custom loyal swa howev inflight wifi better today year ago'
 'unit ua delay winnipeg plane frozen fail make connect ua tightconnect denver'
 'jetblu austin saw water bottl amp snack des

In [119]:
#Convert textual data to numerical data
vectorizer=TfidfVectorizer() # assigns some importance to each word #checks if each word corresponds to +ve or -ve tweets

X_train=vectorizer.fit_transform(X_train) # understands vocab assigns score to each word
X_test=vectorizer.transform(X_test) # ensures test data is transformed using the same vocabulary and IDF values as training data. Ensures consistency and prevents overfitting.

In [120]:
print(X_train)

  (0, 7600)	0.3808401283065275
  (0, 8516)	0.5963696662506563
  (0, 1554)	0.37180137561945403
  (0, 665)	0.32653349233719925
  (0, 2867)	0.3808401283065275
  (0, 5711)	0.2962979531594102
  (0, 9449)	0.1470100436776813
  (1, 2149)	0.3402307791767031
  (1, 1792)	0.6147709621533402
  (1, 3169)	0.21041498245734586
  (1, 6894)	0.3883130452298538
  (1, 9533)	0.4928582981670698
  (1, 4656)	0.26139795902707796
  (2, 2420)	0.9104210369495984
  (2, 9569)	0.4136828924183572
  (3, 9194)	0.2500711092420817
  (3, 9229)	0.4384475695275173
  (3, 5642)	0.35889463746715655
  (3, 3527)	0.1918530532174265
  (3, 1235)	0.23772738736470214
  (3, 2880)	0.40475025981759954
  (3, 5269)	0.5795827850445762
  (3, 290)	0.15284608604942576
  (4, 7396)	0.3645650581185327
  (4, 634)	0.37311953142617965
  :	:
  (13904, 171)	0.2594757038788559
  (13904, 410)	0.5478095689846177
  (13904, 5906)	0.239628807832803
  (13904, 827)	0.2503823136810562
  (13904, 0)	0.4600331392762967
  (13904, 9561)	0.18928108028985657
  (13904,

In [121]:
print(X_test)

  (0, 10106)	0.31282307464548365
  (0, 9569)	0.17710178379753017
  (0, 8975)	0.5864429552538505
  (0, 5471)	0.4070650938712944
  (0, 3824)	0.4333396459170476
  (0, 1235)	0.27560379726353373
  (0, 142)	0.3121196083878299
  (1, 8350)	0.2529308796425997
  (1, 7712)	0.5484309557033089
  (1, 4428)	0.5425713364774974
  (1, 3527)	0.29594698613893317
  (1, 3205)	0.503270487092061
  (2, 7603)	0.876373857586502
  (2, 4656)	0.48163145841914584
  (3, 9569)	0.28176428478535004
  (3, 8841)	0.5653397198577343
  (3, 7521)	0.5501898166188377
  (3, 7376)	0.546160282938542
  (4, 7992)	0.37763397791261605
  (4, 7364)	0.24864437174008366
  (4, 7253)	0.31173986291434463
  (4, 6446)	0.21522989792913275
  (4, 5116)	0.4789169060576912
  (4, 4161)	0.30989516141188617
  (4, 3708)	0.24583542154656143
  :	:
  (728, 603)	0.2661280456817442
  (728, 530)	0.2583407506103422
  (729, 9926)	0.3579512579779954
  (729, 8350)	0.1816220515111129
  (729, 5810)	0.465436254648132
  (729, 4110)	0.23321063967829436
  (729, 4029)	

In [122]:
# Train ML model- Logistic Regression

model=LogisticRegression(max_iter=1000)

In [123]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [124]:
#Model Evaluation on training data

X_train_pred= model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train, X_train_pred)

In [125]:
print('Accuracy of training data:' ,training_data_accuracy)

Accuracy of training data: 0.865832614322692


In [126]:
#Model Evaluation on test data

X_test_pred= model.predict(X_test)
test_data_accuracy= accuracy_score(Y_test, X_test_pred)

In [127]:
print('Accuracy of testing data:' ,test_data_accuracy)


Accuracy of testing data: 0.796448087431694


# Save the Trained Model

In [128]:
import pickle

In [129]:
filename= 'trained_model.sav'
pickle.dump(model, open(filename, 'wb')) #wb-> writing in binary format

In [130]:
#Using saved model for future predictions
loaded_model= pickle.load(open('/content/trained_model.sav', 'rb'))

In [132]:
X_new=X_test[600]
print(Y_test[600])

prediction= loaded_model.predict(X_new)
print(prediction)



negative
['negative']
